# E2B Sandbox 基礎連接測試

## 🎯 實驗目標

驗證E2B Sandbox的基本功能穩定性：
- 連接創建和關閉
- 文件系統操作
- 代碼執行
- 網絡訪問
- 錯誤處理

## ⚠️ 執行前準備

1. 註冊E2B帳號: https://e2b.dev/
2. 獲取API密鑰
3. 安裝依賴: `pip install e2b`

In [1]:
# 安裝必要依賴
!pip install e2b

In [2]:
import asyncio
import os
import json
import time
from datetime import datetime
import getpass

# E2B相關
try:
    from e2b import Sandbox, AsyncSandbox
    print("✅ E2B imported successfully")
    
    # 檢查E2B版本 - 使用替代方法
    try:
        import pkg_resources
        version = pkg_resources.get_distribution("e2b").version
        print(f"📦 E2B版本: {version}")
    except:
        print("📦 E2B已安装但无法获取版本信息")
    
except ImportError:
    print("❌ E2B not installed. Run: pip install e2b")
    raise

✅ E2B imported successfully
📦 E2B版本: 1.7.0


/var/folders/gw/p0n7cj5972g9dg5g16bbrhnm0000gn/T/ipykernel_25154/2817701501.py:15: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## 🔐 安全API配置

In [3]:
# 安全輸入API密鑰
def setup_e2b_api():
    """
    安全地設置E2B API配置
    避免硬編碼敏感信息
    """
    api_key = getpass.getpass("請輸入您的E2B API密鑰: ")
    
    # 設置環境變量
    os.environ['E2B_API_KEY'] = api_key
    
    print("✅ E2B API密鑰配置完成")
    print("🔗 準備連接E2B Sandbox")
    
    return True

#執行配置
setup_e2b_api()  # 取消註釋以輸入API密鑰

請輸入您的E2B API密鑰:  ········


✅ E2B API密鑰配置完成
🔗 準備連接E2B Sandbox


True

## 🧪 測試1: 基礎連接測試

In [4]:
async def test_basic_connection():
    """
    測試E2B Sandbox基礎連接
    
    關鍵驗證點：
    1. Sandbox能否成功創建
    2. 連接是否穩定
    3. 關閉是否正常
    """
    test_results = {
        "test_name": "basic_connection",
        "start_time": datetime.now().isoformat(),
        "success": False,
        "error": None,
        "duration_seconds": 0
    }
    
    start_time = time.time()
    
    try:
        print("🚀 創建E2B Sandbox...")
        
        # 使用AsyncSandbox.create()而不是Sandbox.create()
        from e2b import AsyncSandbox
        sandbox = await AsyncSandbox.create("base")
        print(f"✅ Sandbox創建成功，ID: {sandbox.sandbox_id}")
        
        # 測試基本信息獲取
        print(f"📊 Sandbox狀態: 運行中")
        
        # 使用kill()而不是close()
        await sandbox.kill()
        print("✅ Sandbox已正常關閉")
        
        test_results["success"] = True
        
    except Exception as e:
        print(f"❌ 連接測試失敗: {str(e)}")
        test_results["error"] = str(e)
        
    finally:
        test_results["duration_seconds"] = time.time() - start_time
        test_results["end_time"] = datetime.now().isoformat()
        
    return test_results

# 執行測試
print("⚠️ 請確保已設置API密鑰後執行")
basic_result = await test_basic_connection()
print(f"\n📊 測試結果: {json.dumps(basic_result, indent=2, ensure_ascii=False)}")

⚠️ 請確保已設置API密鑰後執行
🚀 創建E2B Sandbox...
✅ Sandbox創建成功，ID: ip7f4mof1bvugelwvyijz
📊 Sandbox狀態: 運行中
✅ Sandbox已正常關閉

📊 測試結果: {
  "test_name": "basic_connection",
  "start_time": "2025-07-28T15:41:07.457503",
  "success": true,
  "error": null,
  "duration_seconds": 2.266374111175537,
  "end_time": "2025-07-28T15:41:09.723892"
}


In [5]:
async def test_filesystem_operations():
    """
    測試E2B文件系統操作
    
    關鍵驗證點：
    1. 文件寫入
    2. 文件讀取
    3. 目錄操作
    4. 文件權限
    """
    test_results = {
        "test_name": "filesystem_operations",
        "start_time": datetime.now().isoformat(),
        "operations": [],
        "success": False,
        "error": None
    }
    
    try:
        from e2b import AsyncSandbox
        sandbox = await AsyncSandbox.create("base")
        print("✅ Sandbox已創建")
        
        # 測試1: 文件寫入 - 使用files而不是filesystem
        test_content = "Hello E2B from Jupyter!\nTimestamp: " + datetime.now().isoformat()
        await sandbox.files.write("/tmp/test.txt", test_content)
        print("✅ 文件寫入成功: /tmp/test.txt")
        test_results["operations"].append("write_file")
        
        # 測試2: 文件讀取
        read_content = await sandbox.files.read("/tmp/test.txt")
        assert read_content == test_content, "讀取內容與寫入不符"
        print("✅ 文件讀取成功，內容匹配")
        test_results["operations"].append("read_file")
        
        # 測試3: 目錄列表
        files = await sandbox.files.list("/tmp")
        file_names = [f.name for f in files]
        assert "test.txt" in file_names, "創建的文件未在目錄列表中"
        print(f"✅ 目錄列表成功: {len(files)} 個文件")
        test_results["operations"].append("list_directory")
        
        # 測試4: 創建Python腳本
        python_script = '''
import sys
import os
print(f"Python版本: {sys.version}")
print(f"當前目錄: {os.getcwd()}")
print("E2B Python環境測試成功！")
'''
        await sandbox.files.write("/tmp/test_script.py", python_script)
        print("✅ Python腳本創建成功")
        test_results["operations"].append("create_script")
        
        await sandbox.kill()
        test_results["success"] = True
        
    except Exception as e:
        print(f"❌ 文件系統測試失敗: {str(e)}")
        test_results["error"] = str(e)
        try:
            await sandbox.kill()
        except:
            pass
            
    test_results["end_time"] = datetime.now().isoformat()
    return test_results

# 執行測試
filesystem_result = await test_filesystem_operations()
print(f"\n📊 文件系統測試結果: {json.dumps(filesystem_result, indent=2, ensure_ascii=False)}")

✅ Sandbox已創建
✅ 文件寫入成功: /tmp/test.txt
✅ 文件讀取成功，內容匹配
✅ 目錄列表成功: 5 個文件
✅ Python腳本創建成功

📊 文件系統測試結果: {
  "test_name": "filesystem_operations",
  "start_time": "2025-07-28T15:41:09.740942",
  "operations": [
    "write_file",
    "read_file",
    "list_directory",
    "create_script"
  ],
  "success": true,
  "error": null,
  "end_time": "2025-07-28T15:41:12.192918"
}


In [9]:
async def test_code_execution():
    """
    測試E2B代碼執行能力
    
    關鍵驗證點：
    1. Python代碼執行
    2. Shell命令執行
    3. 輸出捕獲
    4. 錯誤處理
    """
    test_results = {
        "test_name": "code_execution", 
        "start_time": datetime.now().isoformat(),
        "executions": [],
        "success": False,
        "error": None
    }
    
    try:
        from e2b import AsyncSandbox
        sandbox = await AsyncSandbox.create("base")
        print("✅ Sandbox已創建")
        
        # 測試1: 簡單Python執行
        result1 = await sandbox.commands.run("python -c 'print(\"Hello from E2B Python!\")'")
        print(f"✅ Python執行成功: {result1.stdout.strip()}")
        test_results["executions"].append({
            "type": "python_simple",
            "exit_code": result1.exit_code,
            "stdout": result1.stdout.strip()
        })
        
        # 測試2: Python腳本文件執行
        script_content = '''
import json
import datetime

data = {
    "message": "E2B Python腳本測試",
    "timestamp": datetime.datetime.now().isoformat(),
    "success": True
}

print(json.dumps(data, ensure_ascii=False))
'''
        await sandbox.files.write("/tmp/test_script.py", script_content)
        result2 = await sandbox.commands.run("python /tmp/test_script.py")
        print(f"✅ Python腳本執行成功")
        
        try:
            script_output = json.loads(result2.stdout.strip())
            print(f"📊 腳本輸出: {script_output['message']}")
        except json.JSONDecodeError:
            print(f"⚠️ 腳本輸出解析失敗: {result2.stdout}")
            
        test_results["executions"].append({
            "type": "python_script",
            "exit_code": result2.exit_code,
            "stdout_length": len(result2.stdout)
        })
        
        # 測試3: Shell命令執行
        result3 = await sandbox.commands.run("ls -la /tmp")
        output_lines = result3.stdout.split('\n')
        print(f"✅ Shell命令執行成功，文件數: {len(output_lines)}")
        test_results["executions"].append({
            "type": "shell_command",
            "exit_code": result3.exit_code,
            "output_lines": len(output_lines)
        })
        
        # 測試4: 錯誤處理 - 預期會失敗，所以捕獲異常
        try:
            result4 = await sandbox.commands.run("python -c 'import nonexistent_module'")
            # 如果沒有拋出異常，檢查exit_code
            if result4.exit_code != 0:
                print(f"✅ 錯誤處理測試: 正確捕獲到錯誤 exit_code={result4.exit_code}")
            else:
                print(f"⚠️ 錯誤處理測試: 未預期的成功")
        except Exception as e:
            print(f"✅ 錯誤處理測試: 成功捕獲異常 - {str(e)[:50]}...")
            result4 = type('MockResult', (), {'exit_code': 1, 'stderr': str(e)})()
            
        test_results["executions"].append({
            "type": "error_handling",
            "exit_code": result4.exit_code,
            "has_stderr": bool(getattr(result4, 'stderr', ''))
        })
        
        await sandbox.kill()
        test_results["success"] = True
        
    except Exception as e:
        print(f"❌ 代碼執行測試失敗: {str(e)}")
        test_results["error"] = str(e)
        try:
            await sandbox.kill()
        except:
            pass
            
    test_results["end_time"] = datetime.now().isoformat()
    return test_results

# 執行測試
execution_result = await test_code_execution()
print(f"\n📊 代碼執行測試結果: {json.dumps(execution_result, indent=2, ensure_ascii=False)}")

✅ Sandbox已創建
✅ Python執行成功: Hello from E2B Python!
✅ Python腳本執行成功
📊 腳本輸出: E2B Python腳本測試
✅ Shell命令執行成功，文件數: 9
✅ 錯誤處理測試: 成功捕獲異常 - Command exited with code 1 and error:
Traceback (m...

📊 代碼執行測試結果: {
  "test_name": "code_execution",
  "start_time": "2025-07-28T15:45:39.056709",
  "executions": [
    {
      "type": "python_simple",
      "exit_code": 0,
      "stdout": "Hello from E2B Python!"
    },
    {
      "type": "python_script",
      "exit_code": 0,
      "stdout_length": 90
    },
    {
      "type": "shell_command",
      "exit_code": 0,
      "output_lines": 9
    },
    {
      "type": "error_handling",
      "exit_code": 1,
      "has_stderr": true
    }
  ],
  "success": true,
  "error": null,
  "end_time": "2025-07-28T15:45:41.392936"
}


In [10]:
async def run_all_tests():
    """
    執行所有基礎測試並生成報告
    """
    print("🚀 開始E2B Sandbox基礎功能完整測試\n")
    print("="*60)
    
    all_results = {
        "test_suite": "e2b_basic_functionality",
        "start_time": datetime.now().isoformat(),
        "tests": [],
        "summary": {
            "total": 0,
            "passed": 0,
            "failed": 0
        }
    }
    
    # 定義測試函數列表
    tests = [
        ("基礎連接", test_basic_connection),
        ("文件系統操作", test_filesystem_operations),
        ("代碼執行", test_code_execution),
        ("網絡訪問", test_network_access)
    ]
    
    # 執行每個測試
    for test_name, test_func in tests:
        print(f"\n🧪 執行測試: {test_name}")
        print("-" * 40)
        
        try:
            result = await test_func()
            all_results["tests"].append(result)
            
            if result["success"]:
                all_results["summary"]["passed"] += 1
                print(f"✅ {test_name} 測試通過")
            else:
                all_results["summary"]["failed"] += 1
                print(f"❌ {test_name} 測試失敗: {result.get('error', '未知錯誤')}")
                
        except Exception as e:
            print(f"❌ {test_name} 測試執行異常: {str(e)}")
            all_results["summary"]["failed"] += 1
            all_results["tests"].append({
                "test_name": test_name.lower().replace(" ", "_"),
                "success": False,
                "error": f"測試執行異常: {str(e)}"
            })
    
    all_results["summary"]["total"] = len(tests)
    all_results["end_time"] = datetime.now().isoformat()
    
    # 生成測試報告
    print("\n" + "="*60)
    print("📊 測試結果摘要")
    print("="*60)
    print(f"總測試數: {all_results['summary']['total']}")
    print(f"通過: {all_results['summary']['passed']}")
    print(f"失敗: {all_results['summary']['failed']}")
    
    success_rate = (all_results['summary']['passed'] / all_results['summary']['total']) * 100
    print(f"成功率: {success_rate:.1f}%")
    
    # 決策建議
    print("\n🎯 建議:")
    if success_rate >= 75:
        print("✅ E2B基礎功能穩定，建議進入下一階段Context Engineering測試")
    elif success_rate >= 50:
        print("⚠️ E2B部分功能可用，建議解決失敗項目後再進入下一階段")
    else:
        print("❌ E2B基礎功能不穩定，建議重新評估技術方案")
    
    return all_results

# 執行完整測試套件
print("⚠️ 請確保已設置E2B API密鑰！")
print("取消下面註釋以執行完整測試:")
test_results = await run_all_tests()

# 保存測試結果
with open("/Users/jason/Developer/AgenticAI_Playground/e2b_experiments/basic_test_results.json", "w") as f:
    json.dump(test_results, f, indent=2, ensure_ascii=False)
print("\n💾 測試結果已保存到: basic_test_results.json")

⚠️ 請確保已設置E2B API密鑰！
取消下面註釋以執行完整測試:
🚀 開始E2B Sandbox基礎功能完整測試


🧪 執行測試: 基礎連接
----------------------------------------
🚀 創建E2B Sandbox...
✅ Sandbox創建成功，ID: i4u2wdkecrwlznhf1objw
📊 Sandbox狀態: 運行中
✅ Sandbox已正常關閉
✅ 基礎連接 測試通過

🧪 執行測試: 文件系統操作
----------------------------------------
✅ Sandbox已創建
✅ 文件寫入成功: /tmp/test.txt
✅ 文件讀取成功，內容匹配
✅ 目錄列表成功: 5 個文件
✅ Python腳本創建成功
✅ 文件系統操作 測試通過

🧪 執行測試: 代碼執行
----------------------------------------
✅ Sandbox已創建
✅ Python執行成功: Hello from E2B Python!
✅ Python腳本執行成功
📊 腳本輸出: E2B Python腳本測試
✅ Shell命令執行成功，文件數: 9
✅ 錯誤處理測試: 成功捕獲異常 - Command exited with code 1 and error:
Traceback (m...
✅ 代碼執行 測試通過

🧪 執行測試: 網絡訪問
----------------------------------------
✅ Sandbox已創建
✅ HTTP請求成功
📊 響應包含 1 個字段
✅ Python網絡請求: 成功! User-Agent: Python-urllib/3.11
✅ DNS解析測試完成: 成功
📊 DNS解析成功: google.com -> 172.253.117.100
✅ 網絡訪問 測試通過

📊 測試結果摘要
總測試數: 4
通過: 4
失敗: 0
成功率: 100.0%

🎯 建議:
✅ E2B基礎功能穩定，建議進入下一階段Context Engineering測試

💾 測試結果已保存到: basic_test_results.json
